In [1]:
#Getting all libraries necessary for now!
import pandas as pd
import numpy as np
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read all these to pandas!
train = pd.read_csv("/content/drive/My Drive/Janata Hack - Agriculture/train_yaOffsB.csv")
test = pd.read_csv("/content/drive/My Drive/Janata Hack - Agriculture/test_pFkWwen.csv")
submission = pd.read_csv("/content/drive/My Drive/Janata Hack - Agriculture/sample_submission_O1oDc4H.csv")


In [4]:
print(train.shape, test.shape)

(88858, 10) (59310, 9)


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88858 entries, 0 to 88857
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       88858 non-null  object 
 1   Estimated_Insects_Count  88858 non-null  int64  
 2   Crop_Type                88858 non-null  int64  
 3   Soil_Type                88858 non-null  int64  
 4   Pesticide_Use_Category   88858 non-null  int64  
 5   Number_Doses_Week        88858 non-null  int64  
 6   Number_Weeks_Used        79858 non-null  float64
 7   Number_Weeks_Quit        88858 non-null  int64  
 8   Season                   88858 non-null  int64  
 9   Crop_Damage              88858 non-null  int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 6.8+ MB


In [6]:
df = train.append(test, ignore_index = True)
print(df.shape)
print(df.head())

(148168, 10)
          ID  Estimated_Insects_Count  Crop_Type  Soil_Type  \
0  F00000001                      188          1          0   
1  F00000003                      209          1          0   
2  F00000004                      257          1          0   
3  F00000005                      257          1          1   
4  F00000006                      342          1          0   

   Pesticide_Use_Category  Number_Doses_Week  Number_Weeks_Used  \
0                       1                  0                0.0   
1                       1                  0                0.0   
2                       1                  0                0.0   
3                       1                  0                0.0   
4                       1                  0                0.0   

   Number_Weeks_Quit  Season  Crop_Damage  
0                  0       1          0.0  
1                  0       2          1.0  
2                  0       2          1.0  
3                  0       2 

In [7]:
df_train=df[df['Crop_Damage'].isnull()==False].copy()
df_test=df[df['Crop_Damage'].isnull()==True].copy()

print(df_train.shape,df_test.shape)


(88858, 10) (59310, 10)


In [8]:
from sklearn.model_selection import StratifiedKFold,train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [9]:
X,Y=df_train.drop(['ID','Crop_Damage'],axis=1),df_train['Crop_Damage']
Xtest=df_test.drop(['ID','Crop_Damage'],axis=1)

print(X.shape,Xtest.shape)
X_train,X_val,Y_train,Y_val = train_test_split(X,Y,test_size=0.25,random_state = 2020)

(88858, 8) (59310, 8)


In [10]:
LG = LGBMClassifier(n_estimators=3000,random_state=2020,learning_rate=0.03,colsample_bytree=0.2,objective='categorical',scale_pos_weight=1)
LG.fit(X_train, Y_train)
y_pred = LG.predict(X_val)
print('Accuracy Score:', accuracy_score(Y_val, y_pred))

Accuracy Score: 0.8431690299347288


In [14]:
LGBMerror=[]
y_pred_LGBM=[]
fold=StratifiedKFold(n_splits=8,shuffle=True,random_state=2020)
i=1
for train_index, test_index in fold.split(X,Y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    model=LGBMClassifier(n_estimators=3000,random_state=2020,learning_rate=0.03,colsample_bytree=0.2,objective='categorical',scale_pos_weight=1)
    model.fit(X_train,Y_train,eval_set=[(X_test, Y_test)], early_stopping_rounds=5000, verbose=200)
    preds=model.predict(X_test)[:]
    print("LGBMerror: ",accuracy_score(Y_test,preds))
    LGBMerror.append(accuracy_score(Y_test,preds))
    p = model.predict(Xtest)[:]
    i=i+1
    y_pred_LGBM.append(p)

Training until validation scores don't improve for 5000 rounds.
[200]	valid_0's multi_logloss: 0.443599
[400]	valid_0's multi_logloss: 0.433797
[600]	valid_0's multi_logloss: 0.431493
[800]	valid_0's multi_logloss: 0.430864
[1000]	valid_0's multi_logloss: 0.430671
[1200]	valid_0's multi_logloss: 0.430602
[1400]	valid_0's multi_logloss: 0.430584
[1600]	valid_0's multi_logloss: 0.430566
[1800]	valid_0's multi_logloss: 0.430525
[2000]	valid_0's multi_logloss: 0.430516
[2200]	valid_0's multi_logloss: 0.430516
[2400]	valid_0's multi_logloss: 0.430534
[2600]	valid_0's multi_logloss: 0.430532
[2800]	valid_0's multi_logloss: 0.430551
[3000]	valid_0's multi_logloss: 0.430573
Did not meet early stopping. Best iteration is:
[1863]	valid_0's multi_logloss: 0.43051
LGBMerror:  0.8428159884767735
Training until validation scores don't improve for 5000 rounds.
[200]	valid_0's multi_logloss: 0.443186
[400]	valid_0's multi_logloss: 0.432955
[600]	valid_0's multi_logloss: 0.430406
[800]	valid_0's multi_

In [15]:
submission['Crop_Damage'] = np.mean(y_pred_LGBM,0)

In [16]:
submission.to_csv("/content/drive/My Drive/Janata Hack - Agriculture/Submission_LGBM.csv", index= False)
submission.shape

(59310, 2)